# 附录 10.3: 搜索与检索 (Search & Retrieval)

## 概述
您知道可以使用 Claude 来**为您搜索维基百科**吗？Claude 可以查找和检索文章，然后您还可以使用 Claude 来总结和综合这些文章，从找到的内容中编写新颖的内容，等等。不仅仅是维基百科！您还可以搜索自己的文档，无论是存储为纯文本还是嵌入在向量数据库中。

## 什么是 RAG (检索增强生成)?
RAG (Retrieval-Augmented Generation) 是一种将外部知识检索与大语言模型生成能力相结合的技术：

### RAG 的核心组件：
1. **检索器 (Retriever)**: 从知识库中找到相关信息
2. **生成器 (Generator)**: 基于检索到的信息生成回答
3. **知识库 (Knowledge Base)**: 存储待检索的文档和数据

### RAG 的优势：
- **实时性**: 能够获取最新信息，不受模型训练时间限制
- **准确性**: 基于真实数据源，减少幻觉现象
- **可控性**: 可以指定特定的知识源
- **可扩展性**: 随时更新知识库内容

## 学习资源

查看我们的 [RAG 实践教程示例](https://github.com/anthropics/anthropic-cookbook/blob/main/third_party/Wikipedia/wikipedia-search-cookbook.ipynb) 来学习如何通过从向量数据库、维基百科、互联网等检索的数据来补充 Claude 的知识，提高 Claude 响应的准确性和相关性。在那里，您还可以学习如何使用某些 [嵌入技术 (embeddings)](https://docs.anthropic.com/claude/docs/embeddings) 和向量数据库工具。

如果您有兴趣了解使用 Claude 的高级 RAG 架构，请查看我们的 [Claude 3 RAG 架构技术演示幻灯片](https://docs.google.com/presentation/d/1zxkSI7lLUBrZycA-_znwqu8DDyVhHLkQGScvzaZrUns/edit#slide=id.g2c736259dac_63_782)。

## 技术架构图
```
用户查询 → 向量检索 → 相关文档 → Claude处理 → 生成回答
    ↓           ↓          ↓         ↓         ↓
  Query    Embeddings   Context   Generate  Response
```

## 后续章节预告
在接下来的示例中，我们将演示：
- 如何设置向量数据库
- 实现文档嵌入和检索
- 与 Claude API 集成
- 构建完整的 RAG 应用

In [ ]:
# 环境设置和依赖安装
# 首先安装必要的依赖包

# !pip install anthropic wikipedia-api sentence-transformers faiss-cpu numpy

# 导入必要的库
import anthropic
import wikipedia
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import json
from typing import List, Dict, Any
import os

# 设置 Anthropic API 密钥
# 请确保在环境变量中设置 ANTHROPIC_API_KEY
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY")  # 从环境变量获取API密钥
)

print("✅ 环境设置完成！")


In [ ]:
class WikipediaSearcher:
    """
    维基百科搜索器类
    用于搜索、获取和处理维基百科文章内容
    """
    
    def __init__(self, language='zh'):
        """
        初始化维基百科搜索器
        
        Args:
            language (str): 维基百科语言版本，默认为中文 'zh'
        """
        wikipedia.set_lang(language)
        self.language = language
    
    def search_articles(self, query: str, max_results: int = 5) -> List[str]:
        """
        搜索维基百科文章标题
        
        Args:
            query (str): 搜索关键词
            max_results (int): 最大返回结果数量
            
        Returns:
            List[str]: 文章标题列表
        """
        try:
            # 搜索相关文章标题
            titles = wikipedia.search(query, results=max_results)
            print(f"🔍 找到 {len(titles)} 篇相关文章:")
            for i, title in enumerate(titles, 1):
                print(f"  {i}. {title}")
            return titles
        except Exception as e:
            print(f"❌ 搜索失败: {e}")
            return []
    
    def get_article_content(self, title: str, max_sentences: int = 10) -> Dict[str, Any]:
        """
        获取维基百科文章内容
        
        Args:
            title (str): 文章标题
            max_sentences (int): 最大返回句子数量
            
        Returns:
            Dict[str, Any]: 包含标题、摘要、内容等信息的字典
        """
        try:
            # 获取文章页面
            page = wikipedia.page(title)
            
            # 将内容分割为句子并限制数量
            sentences = page.content.split('. ')[:max_sentences]
            limited_content = '. '.join(sentences)
            
            article_info = {
                'title': page.title,
                'url': page.url,
                'summary': page.summary,
                'content': limited_content,
                'length': len(page.content)
            }
            
            print(f"📄 成功获取文章: {page.title}")
            print(f"📊 文章长度: {len(page.content)} 字符")
            
            return article_info
            
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"⚠️  存在歧义，可选择: {e.options[:5]}")
            # 选择第一个选项
            return self.get_article_content(e.options[0], max_sentences)
        except Exception as e:
            print(f"❌ 获取文章失败: {e}")
            return {}

# 创建维基百科搜索器实例
wiki_searcher = WikipediaSearcher()

# 示例：搜索关于人工智能的文章
search_query = "人工智能"
articles = wiki_searcher.search_articles(search_query, max_results=3)


In [ ]:
class ClaudeRAGProcessor:
    """
    Claude RAG 处理器
    结合维基百科搜索和 Claude 分析能力
    """
    
    def __init__(self, wiki_searcher: WikipediaSearcher):
        """
        初始化 Claude RAG 处理器
        
        Args:
            wiki_searcher (WikipediaSearcher): 维基百科搜索器实例
        """
        self.wiki_searcher = wiki_searcher
        
    def search_and_analyze(self, user_query: str, max_articles: int = 3) -> str:
        """
        搜索相关文章并使用 Claude 进行分析
        
        Args:
            user_query (str): 用户查询
            max_articles (int): 最大搜索文章数量
            
        Returns:
            str: Claude 分析后的响应
        """
        print(f"🚀 开始处理查询: {user_query}")
        
        # 1. 搜索相关文章
        article_titles = self.wiki_searcher.search_articles(user_query, max_articles)
        
        if not article_titles:
            return "❌ 未找到相关文章"
        
        # 2. 获取文章内容
        articles_content = []
        for title in article_titles[:max_articles]:
            article = self.wiki_searcher.get_article_content(title, max_sentences=8)
            if article:
                articles_content.append(article)
        
        # 3. 构建 Claude 提示词
        context = self._build_context(articles_content)
        prompt = self._build_prompt(user_query, context)
        
        # 4. 调用 Claude API
        try:
            response = client.messages.create(
                model="claude-3-sonnet-20240229",  # 使用 Claude 3 Sonnet 模型
                max_tokens=1500,
                temperature=0.3,  # 较低的温度以确保准确性
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            )
            
            return response.content[0].text
            
        except Exception as e:
            return f"❌ Claude API 调用失败: {e}"
    
    def _build_context(self, articles: List[Dict[str, Any]]) -> str:
        """
        构建上下文信息
        
        Args:
            articles (List[Dict]): 文章列表
            
        Returns:
            str: 格式化的上下文字符串
        """
        context_parts = []
        
        for i, article in enumerate(articles, 1):
            context_part = f"""
【文章 {i}】
标题: {article['title']}
链接: {article['url']}
摘要: {article['summary']}
内容节选: {article['content'][:1000]}...
"""
            context_parts.append(context_part)
        
        return "\n".join(context_parts)
    
    def _build_prompt(self, user_query: str, context: str) -> str:
        """
        构建 Claude 提示词
        
        Args:
            user_query (str): 用户查询
            context (str): 上下文信息
            
        Returns:
            str: 完整的提示词
        """
        prompt = f"""
你是一个知识助手。基于以下从维基百科检索到的信息，请回答用户的问题。

用户问题：{user_query}

参考资料：
{context}

请根据上述资料回答用户问题，要求：
1. 回答要准确、全面
2. 如果资料中有具体数据或事实，请引用
3. 保持客观和中立的语调
4. 如果资料不足以完全回答问题，请说明
5. 在回答末尾提供参考文章的链接

回答：
"""
        return prompt

# 创建 Claude RAG 处理器
rag_processor = ClaudeRAGProcessor(wiki_searcher)

# 示例查询
user_question = "什么是机器学习？它有哪些主要应用领域？"
print(f"用户问题: {user_question}")
print("-" * 50)


In [ ]:
# 执行 RAG 查询示例
# 注意：需要设置 ANTHROPIC_API_KEY 环境变量才能运行

# 取消注释下面的代码来运行示例
# result = rag_processor.search_and_analyze(user_question)
# print("Claude 的回答：")
# print(result)

# 模拟输出示例
print("Claude 的回答：")
print("""
基于维基百科的资料，我来回答您关于机器学习的问题：

**什么是机器学习？**

机器学习是人工智能的一个分支，它是一种让计算机在没有明确编程的情况下学习的方法。机器学习算法通过分析数据来识别模式，并做出预测或决策。

**主要应用领域：**

1. **图像识别**: 人脸识别、医学影像分析、自动驾驶汽车的视觉系统
2. **自然语言处理**: 机器翻译、语音识别、聊天机器人
3. **推荐系统**: 电商平台、流媒体服务的个性化推荐
4. **金融服务**: 风险评估、欺诈检测、算法交易
5. **医疗健康**: 疾病诊断、药物发现、基因分析
6. **交通运输**: 自动驾驶、交通优化、路线规划

参考资料：
- 机器学习 - https://zh.wikipedia.org/wiki/机器学习
- 人工智能 - https://zh.wikipedia.org/wiki/人工智能
- 深度学习 - https://zh.wikipedia.org/wiki/深度学习
""")


In [ ]:
class VectorSearchEngine:
    """
    向量搜索引擎
    使用语义嵌入进行文档检索
    """
    
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        初始化向量搜索引擎
        
        Args:
            model_name (str): 句子嵌入模型名称
        """
        self.encoder = SentenceTransformer(model_name)
        self.documents = []  # 存储文档
        self.embeddings = None  # 存储文档嵌入
        self.index = None  # FAISS 索引
        
        print(f"✅ 向量搜索引擎初始化完成，使用模型: {model_name}")
    
    def add_documents(self, documents: List[Dict[str, Any]]):
        """
        添加文档到搜索引擎
        
        Args:
            documents (List[Dict]): 文档列表，每个文档包含 'title', 'content' 等字段
        """
        print(f"📚 正在添加 {len(documents)} 个文档...")
        
        # 存储文档
        self.documents.extend(documents)
        
        # 准备用于嵌入的文本
        texts_for_embedding = []
        for doc in documents:
            # 组合标题和内容用于嵌入
            text = f"{doc.get('title', '')} {doc.get('content', '')}".strip()
            texts_for_embedding.append(text)
        
        # 生成嵌入向量
        print("🔄 正在生成文档嵌入向量...")
        new_embeddings = self.encoder.encode(texts_for_embedding)
        
        # 更新嵌入矩阵
        if self.embeddings is None:
            self.embeddings = new_embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, new_embeddings])
        
        # 重建 FAISS 索引
        self._build_index()
        
        print(f"✅ 文档添加完成，当前共有 {len(self.documents)} 个文档")
    
    def _build_index(self):
        """构建 FAISS 索引用于快速相似度搜索"""
        if self.embeddings is not None:
            dimension = self.embeddings.shape[1]
            self.index = faiss.IndexFlatIP(dimension)  # 使用内积相似度
            
            # 归一化嵌入向量
            faiss.normalize_L2(self.embeddings)
            self.index.add(self.embeddings)
            
            print(f"🔍 FAISS 索引构建完成，维度: {dimension}")
    
    def search(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """
        语义搜索文档
        
        Args:
            query (str): 查询文本
            top_k (int): 返回最相似的文档数量
            
        Returns:
            List[Dict]: 搜索结果，包含文档和相似度分数
        """
        if self.index is None or len(self.documents) == 0:
            print("❌ 搜索引擎为空，请先添加文档")
            return []
        
        print(f"🔍 搜索查询: '{query}'")
        
        # 对查询进行嵌入
        query_embedding = self.encoder.encode([query])
        faiss.normalize_L2(query_embedding)
        
        # 执行搜索
        scores, indices = self.index.search(query_embedding, top_k)
        
        # 整理搜索结果
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx < len(self.documents):  # 确保索引有效
                result = {
                    'rank': i + 1,
                    'score': float(score),
                    'document': self.documents[idx]
                }
                results.append(result)
        
        print(f"📊 找到 {len(results)} 个相关文档")
        return results
    
    def print_search_results(self, results: List[Dict[str, Any]]):
        """
        打印搜索结果
        
        Args:
            results (List[Dict]): 搜索结果
        """
        for result in results:
            print(f"\n排名 {result['rank']} (相似度: {result['score']:.3f})")
            print(f"标题: {result['document'].get('title', 'N/A')}")
            print(f"内容摘要: {result['document'].get('content', 'N/A')[:200]}...")
            if 'url' in result['document']:
                print(f"链接: {result['document']['url']}")

# 创建向量搜索引擎实例
vector_engine = VectorSearchEngine()

# 示例：准备一些文档数据
sample_documents = [
    {
        'title': '深度学习基础',
        'content': '深度学习是机器学习的一个分支，它使用多层神经网络来学习数据的表示。深度学习在图像识别、自然语言处理和语音识别等领域取得了突破性进展。',
        'category': 'AI技术',
        'url': 'https://example.com/deep-learning'
    },
    {
        'title': '自然语言处理技术',
        'content': '自然语言处理(NLP)是人工智能领域的一个重要分支，旨在让计算机理解和生成人类语言。包括文本分析、机器翻译、情感分析等应用。',
        'category': 'AI应用',
        'url': 'https://example.com/nlp'
    },
    {
        'title': '计算机视觉原理',
        'content': '计算机视觉是让计算机获得类似人类视觉系统的能力，包括图像识别、目标检测、图像分割等技术。广泛应用于自动驾驶、医疗诊断等领域。',
        'category': 'AI应用',
        'url': 'https://example.com/cv'
    }
]

print("准备添加示例文档到向量搜索引擎...")


In [ ]:
# 演示向量搜索功能
# 注意：实际运行需要安装 sentence-transformers 和 faiss-cpu

# 取消注释下面的代码来运行向量搜索示例
# vector_engine.add_documents(sample_documents)

# 执行不同类型的语义搜索
# search_queries = [
#     "如何让机器理解人类语言？",
#     "图像识别相关技术",
#     "神经网络和深度学习"
# ]

# for query in search_queries:
#     print(f"\n{'='*50}")
#     results = vector_engine.search(query, top_k=2)
#     vector_engine.print_search_results(results)

# 模拟向量搜索输出
print("向量搜索引擎演示：")
print("="*50)

print("\n🔍 搜索查询: '如何让机器理解人类语言？'")
print("📊 找到 2 个相关文档")
print("\n排名 1 (相似度: 0.758)")
print("标题: 自然语言处理技术")
print("内容摘要: 自然语言处理(NLP)是人工智能领域的一个重要分支，旨在让计算机理解和生成人类语言。包括文本分析、机器翻译、情感分析等应用。")
print("链接: https://example.com/nlp")

print("\n排名 2 (相似度: 0.632)")
print("标题: 深度学习基础")
print("内容摘要: 深度学习是机器学习的一个分支，它使用多层神经网络来学习数据的表示。深度学习在图像识别、自然语言处理和语音识别等领域取得了突破性进展。")
print("链接: https://example.com/deep-learning")

print("\n" + "="*50)
print("\n🔍 搜索查询: '图像识别相关技术'")
print("📊 找到 2 个相关文档")
print("\n排名 1 (相似度: 0.812)")
print("标题: 计算机视觉原理")
print("内容摘要: 计算机视觉是让计算机获得类似人类视觉系统的能力，包括图像识别、目标检测、图像分割等技术。广泛应用于自动驾驶、医疗诊断等领域。")
print("链接: https://example.com/cv")

print("\n排名 2 (相似度: 0.689)")
print("标题: 深度学习基础")  
print("内容摘要: 深度学习是机器学习的一个分支，它使用多层神经网络来学习数据的表示。深度学习在图像识别、自然语言处理和语音识别等领域取得了突破性进展。")
print("链接: https://example.com/deep-learning")
